In [ ]:
Name: Umayer Kabir
UTA ID: 1002173764

# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [8]:
# Q1 - Solution Cell

class Matrix:
    

    def __init__(self, a, b=None):
        # Two ways to initialize:
        # 1) Matrix(n, m) -> creates n x m zero matrix
        # 2) Matrix([[...], [...], ...]) with list of lists
        
        if b is None:
            # assume list of lists
            if not isinstance(a, list):
                raise TypeError("Matrix(data) expects a list of lists.")
            
            # make a deep copy so outside changes don't affect this
            self.data = [row[:] for row in a]
        
        else:
            # Matrix(n, m)
            n, m = a, b
            if not (isinstance(n, int) and isinstance(m, int) and n >= 0 and m >= 0):
                raise ValueError("Matrix(n,m) expects nonnegative integers.")
            
            self.data = [[0.0 for _ in range(m)] for _ in range(n)]

    # SHAPE METHOD 
    def shape(self):
        # rows = number of rows
        n = len(self.data)
        # columns = maximum row length (handles uneven rows safely)
        m = max((len(row) for row in self.data), default=0)
        return (n, m)

    # INDEXING
    # Allows M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        return self.data[key]

    def __setitem__(self, key, value):
        # Allows M[i, j] = value
        if not isinstance(key, tuple) or len(key) != 2:
            raise TypeError("Use M[i, j] = value for setting.")
        i, j = key
        self.data[i][j] = float(value)

    # ASSIGN METHOD
    def assign(self, other):
        """
        Copies values from another matrix.
        Sizes must match.
        """
        if not isinstance(other, Matrix):
            raise TypeError("assign expects another Matrix.")

        if self.shape() != other.shape():
            raise ValueError("Cannot assign: shapes differ.")

        n, m = self.shape()
        for i in range(n):
            for j in range(m):
                self.data[i][j] = other.data[i][j]

        return self

    def __repr__(self):
        return f"Matrix({self.data})"

In [9]:
# Q1 - Test Cell

# initialize with (n,m)
M = Matrix(2, 3)
print("M:", M)
print("M shape should be (2,3):", len(M.data), len(M.data[0]))

# initialize with list of lists
A = Matrix([[1, 2], [3, 4]])
print("A:", A)

# indexing: M[i][j]
print("A[1][0] should be 3:", A[1][0])

# indexing: M[i,j]
print("A[0,1] should be 2:", A[0, 1])

# setting an element
A[0, 0] = 99
print("A after setting A[0,0]=99:", A)

B = Matrix([[5, 6], [7, 8]])
A.assign(B)
print("A after assign(B) should match B:", A)

M: Matrix([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
M shape should be (2,3): 2 3
A: Matrix([[1, 2], [3, 4]])
A[1][0] should be 3: 3
A[0,1] should be 2: 2
A after setting A[0,0]=99: Matrix([[99.0, 2], [3, 4]])
A after assign(B) should match B: Matrix([[5, 6], [7, 8]])


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * Modify `__getitem__` implemented above to support slicing.
        

In [12]:
# Q2 - Solution Cell

class Matrix(Matrix):
    def transpose(self):
        # transpose swaps rows/cols
        n, m = self.shape()
        out = []
        for j in range(m):
            new_row = []
            for i in range(n):
                # if the matrix is ragged(rows don't contain same number), missing spots act like 0
                if j < len(self.data[i]):
                    new_row.append(self.data[i][j])
                else:
                    new_row.append(0.0)
            out.append(new_row)
        return Matrix(out)

    def row(self, i):
        # returns i-th row as a 1 x m matrix
        return Matrix([self.data[i][:]])

    def column(self, j):
        # returns j-th column as an n x 1 matrix
        n, m = self.shape()
        out = []
        for i in range(n):
            val = self.data[i][j] if j < len(self.data[i]) else 0.0
            out.append([val])
        return Matrix(out)

    def to_list(self):
        # just returns a deep copy
        return [row[:] for row in self.data]

    def block(self, n0, n1, m0, m1):
        """
        Returns submatrix:
          rows    m0 .. m1-1
          cols    n0 .. n1-1
        """
        n, m = self.shape()   # (rows, cols)
        if not (0 <= m0 <= m1 <= n):
            raise ValueError("Row range out of bounds.")
        if not (0 <= n0 <= n1 <= m):
            raise ValueError("Column range out of bounds.")

        out = []
        for i in range(m0, m1):
            row = []
            for j in range(n0, n1):
                row.append(self.data[i][j] if j < len(self.data[i]) else 0.0)
            out.append(row)
        return Matrix(out)

    # Modifying __getitem__ to support slicing like M[r0:r1, c0:c1]
    def __getitem__(self, key):
        # if it isn't a tuple, behave like normal (M[i] gives a row list)
        if not isinstance(key, tuple):
            return self.data[key]

        r_key, c_key = key

        # if no slicing, just return single element
        if not isinstance(r_key, slice) and not isinstance(c_key, slice):
            return self.data[r_key][c_key]

        # figure row slice bounds
        n_rows = len(self.data)
        if isinstance(r_key, slice):
            r0, r1, rs = r_key.indices(n_rows)
        else:
            r0, r1, rs = r_key, r_key + 1, 1

        # figure col slice bounds using matrix "max cols"
        max_cols = self.shape()[1]
        if isinstance(c_key, slice):
            c0, c1, cs = c_key.indices(max_cols)
        else:
            c0, c1, cs = c_key, c_key + 1, 1

        # keeping it simple: no step slicing
        if rs != 1 or cs != 1:
            raise ValueError("Step slicing not supported.")

        return self.block(c0, c1, r0, r1)

In [13]:
# Q2 - Test Cell

A = Matrix([[1, 2, 3],
            [4, 5, 6]])

print("A:", A)
print("A.shape() should be (2,3):", A.shape())

AT = A.transpose()
print("AT should be 3x2:", AT)
print("AT.shape() should be (3,2):", AT.shape())

print("Row 0 should be [[1,2,3]]:", A.row(0))
print("Column 1 should be [[2],[5]]:", A.column(1))

print("to_list should be [[1,2,3],[4,5,6]]:", A.to_list())

# block(cols 1..3, rows 0..2) -> [[2,3],[5,6]]
print("block(1,3,0,2) should be [[2,3],[5,6]]:", A.block(1, 3, 0, 2))

# slicing test (same as block above)
print("A[0:2, 1:3] should match block:", A[0:2, 1:3])

# also test single element with tuple indexing
print("A[1,2] should be 6:", A[1, 2])

A: Matrix([[1, 2, 3], [4, 5, 6]])
A.shape() should be (2,3): (2, 3)
AT should be 3x2: Matrix([[1, 4], [2, 5], [3, 6]])
AT.shape() should be (3,2): (3, 2)
Row 0 should be [[1,2,3]]: Matrix([[1, 2, 3]])
Column 1 should be [[2],[5]]: Matrix([[2], [5]])
to_list should be [[1,2,3],[4,5,6]]: [[1, 2, 3], [4, 5, 6]]
block(1,3,0,2) should be [[2,3],[5,6]]: Matrix([[2, 3], [5, 6]])
A[0:2, 1:3] should match block: Matrix([[2, 3], [5, 6]])
A[1,2] should be 6: 6


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [14]:
# Q3 - Solution Cell

def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    I = zeros(n, n)
    for i in range(n):
        I[i, i] = 1.0
    return I

In [15]:
# Q3 - Test Cell

print("constant(2,3,7):", constant(2, 3, 7))
print("zeros(2,2):", zeros(2, 2))
print("ones(2,4):", ones(2, 4))
print("eye(3):", eye(3))

constant(2,3,7): Matrix([[7.0, 7.0, 7.0], [7.0, 7.0, 7.0]])
zeros(2,2): Matrix([[0.0, 0.0], [0.0, 0.0]])
ones(2,4): Matrix([[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0]])
eye(3): Matrix([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [16]:
# Q4 - Solution Cell

class Matrix(Matrix):
    def _check_same_shape(self, other):
        if self.shape() != other.shape():
            raise ValueError(f"Shape mismatch: {self.shape()} vs {other.shape()}")

    def scalar_mul(self, c):
        c = float(c)
        n, m = self.shape()
        out = []
        for i in range(n):
            row = []
            for j in range(m):
                val = self.data[i][j] if j < len(self.data[i]) else 0.0
                row.append(c * val)
            out.append(row)
        return Matrix(out)

    def add(self, N):
        self._check_same_shape(N)
        n, m = self.shape()
        out = []
        for i in range(n):
            row = []
            for j in range(m):
                a = self.data[i][j] if j < len(self.data[i]) else 0.0
                b = N.data[i][j] if j < len(N.data[i]) else 0.0
                row.append(a + b)
            out.append(row)
        return Matrix(out)

    def sub(self, N):
        self._check_same_shape(N)
        n, m = self.shape()
        out = []
        for i in range(n):
            row = []
            for j in range(m):
                a = self.data[i][j] if j < len(self.data[i]) else 0.0
                b = N.data[i][j] if j < len(N.data[i]) else 0.0
                row.append(a - b)
            out.append(row)
        return Matrix(out)

    def element_mult(self, N):
        self._check_same_shape(N)
        n, m = self.shape()
        out = []
        for i in range(n):
            row = []
            for j in range(m):
                a = self.data[i][j] if j < len(self.data[i]) else 0.0
                b = N.data[i][j] if j < len(N.data[i]) else 0.0
                row.append(a * b)
            out.append(row)
        return Matrix(out)

    def mat_mult(self, N):
        # matrix product (n x m) times (m x p) => (n x p)
        n, m = self.shape()
        n2, p = N.shape()
        if m != n2:
            raise ValueError(f"Cannot multiply shapes {self.shape()} and {N.shape()}")

        out = []
        for i in range(n):
            row = []
            for j in range(p):
                s = 0.0
                for k in range(m):
                    a = self.data[i][k] if k < len(self.data[i]) else 0.0
                    b = N.data[k][j] if j < len(N.data[k]) else 0.0
                    s += a * b
                row.append(s)
            out.append(row)
        return Matrix(out)

    def equals(self, N):
        if not isinstance(N, Matrix):
            return False
        if self.shape() != N.shape():
            return False
        n, m = self.shape()
        for i in range(n):
            for j in range(m):
                a = self.data[i][j] if j < len(self.data[i]) else 0.0
                b = N.data[i][j] if j < len(N.data[i]) else 0.0
                if a != b:
                    return False
        return True

In [17]:
# Q4 - Test Cell

A = Matrix([[1, 2], [3, 4]])
B = Matrix([[10, 20], [30, 40]])

print("A + B:", A.add(B))
print("B - A:", B.sub(A))
print("2 * A (scalar_mul):", A.scalar_mul(2))
print("A element_mult B:", A.element_mult(B))

C = Matrix([[1, 0, 2],
            [0, 1, 3]])
D = Matrix([[1, 2],
            [3, 4],
            [5, 6]])
print("C mat_mult D should be 2x2:", C.mat_mult(D), "shape:", C.mat_mult(D).shape())

print("A equals A:", A.equals(A))
print("A equals B:", A.equals(B))

A + B: Matrix([[11, 22], [33, 44]])
B - A: Matrix([[9, 18], [27, 36]])
2 * A (scalar_mul): Matrix([[2.0, 4.0], [6.0, 8.0]])
A element_mult B: Matrix([[10, 40], [90, 160]])
C mat_mult D should be 2x2: Matrix([[11.0, 14.0], [18.0, 22.0]]) shape: (2, 2)
A equals A: True
A equals B: False


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [18]:
# Q5 - Solution Cell

class Matrix(Matrix):
    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __mul__(self, other):
        # If it's a number: scalar multiply
        if isinstance(other, (int, float)):
            return self.scalar_mul(other)
        # If it's another Matrix: matrix product
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        return NotImplemented

    def __rmul__(self, other):
        # handle 2 * M
        if isinstance(other, (int, float)):
            return self.scalar_mul(other)
        return NotImplemented

    def __eq__(self, other):
        return self.equals(other)

    def __ne__(self, other):
        return not self.equals(other)

In [19]:
# Q5 - Test Cell

M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

print("2*M:", 2 * M)
print("M*2:", M * 2)
print("M+N:", M + N)
print("N-M:", N - M)

# matrix multiply
print("M*N:", M * N)

print("M==M should be True:", M == M)
print("M!=N should be True:", M != N)

2*M: Matrix([[2.0, 4.0], [6.0, 8.0]])
M*2: Matrix([[2.0, 4.0], [6.0, 8.0]])
M+N: Matrix([[6, 8], [10, 12]])
N-M: Matrix([[4, 4], [4, 4]])
M*N: Matrix([[19.0, 22.0], [43.0, 50.0]])
M==M should be True: True
M!=N should be True: True


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [21]:
# Q6 - Solution Cell

A = Matrix([[1, 2],
            [3, 4]])
B = Matrix([[2, 0],
            [1, 2]])
C = Matrix([[0, 1],
            [5, 6]])

I = eye(2)

In [22]:
# Q6 - Test Cell
# (AB)C = A(BC)

left1  = (A * B) * C
right1 = A * (B * C)
print("(AB)C:", left1)
print("A(BC):", right1)
print("(AB)C == A(BC) ?", left1 == right1)

print()

# A(B + C) = AB + AC
left2  = A * (B + C)
right2 = (A * B) + (A * C)
print("A(B+C):", left2)
print("AB+AC:", right2)
print("A(B+C) == AB+AC ?", left2 == right2)

print()

# AB != BA (usually)
AB = A * B
BA = B * A
print("AB:", AB)
print("BA:", BA)
print("AB != BA ?", AB != BA)

print()

# AI = A
AI = A * I
print("A*I:", AI)
print("A*I == A ?", AI == A)

(AB)C: Matrix([[20.0, 28.0], [40.0, 58.0]])
A(BC): Matrix([[20.0, 28.0], [40.0, 58.0]])
(AB)C == A(BC) ? True

A(B+C): Matrix([[14.0, 17.0], [30.0, 35.0]])
AB+AC: Matrix([[14.0, 17.0], [30.0, 35.0]])
A(B+C) == AB+AC ? True

AB: Matrix([[4.0, 4.0], [10.0, 8.0]])
BA: Matrix([[2.0, 4.0], [7.0, 10.0]])
AB != BA ? True

A*I: Matrix([[1.0, 2.0], [3.0, 4.0]])
A*I == A ? True
